In [5]:
available_fields = {
    1: ("ALL", "All Fields"),
    2: ("TR.CompanyMarketCap", "Company Market Cap"),
    3: ("TR.Revenue", "Revenue"),
    4: ("TR.TotalEquity", "Total Equity"),
    5: ("TR.F.ReturnAvgComEqPctTTM", "Return on Average Common Equity - %, TTM"),
    6: ("TR.F.ReturnAvgTotAssetsPctTTM", "Return on Average Total Assets - %, TTM"),
    7: ("TR.F.PriceToBookValuePerShrIssue", "Price to Book Value per Share - Issue"),
    8: ("TR.WACCBeta", "Beta"),
    9: ("TR.F.CashCashEquivTotPctOfTotAssets", "Cash & Cash Equivalents - Total Percentage of Total Assets"),
    10: ("TR.F.TotCashSTInvstPctOfTotAssets", "Total Cash & ST Investments Percentage of Total Assets"),
    11: ("TR.F.RcvblPctOfTotAssets", "Receivables Percentage of Total Assets"),
    12: ("TR.F.InvntPctOfTotAssets", "Inventory Percentage of Total Assets"),
    13: ("TR.BIDPRICE", "Bid Price"),
    14: ("TR.F.QuickRatio", "Quick Ratio"),
    15: ("TR.ASKPRICE", "Ask Price"),
    16: ("TR.F.PriceToEBITDAPerShr", "Price to EBITDA per Share"),
    17: ("TR.F.STDebtCurrPortOfLTDebt", "Short-Term Debt Current Portion of Long-Term Debt"),
    18: ("TR.F.WkgCapPctTotLTCap", "Working Capital Percentage of Long-Term Capital"),
    19: ("TR.F.WkgCapToTotAssets", "Working Capital to Total Assets"),
    20: ("TR.F.WkgCapYoYAvg", "Working Capital Year over Year Average"),
    21: ("TR.F.NetPPEPctOfTotAssets", "Net Property, Plant & Equipment Percentage of Total Assets"),
    22: ("TR.F.GoodwNetPctOfTotAssets", "Goodwill - Net Percentage of Total Assets"),
    23: ("TR.F.IntangNetPctOfTotAssets", "Intangibles - Net Percentage of Total Assets"),
    24: ("TR.F.RetainedEarnToTotAssets", "Retained Earnings to Total Assets"),
    25: ("TR.F.WkgCapNonCashPerShr", "Working Capital - Non-Cash per Share"),
    26: ("TR.F.WkgCapPerShr", "Working Capital per Share"),
    27: ("TR.F.WkgCapNonCashYoYPctChg", "Working Capital - Non-Cash Year over Year Percentage Change"),
    28: ("TR.F.WkgCap5YrAvg", "Working Capital 5-Year Average"),
    29: ("TR.F.TotRevenue", "Total Revenue"),
    30: ("TR.F.PriceToNetCFOpPerShr", "Price to Net Cash Flow from Operations per Share"),
    31: ("TR.F.PriceToFOCFPerShr", "Price to Free Cash Flow per Share"),
    32: ("TR.F.PriceToLeveredFOCFPerShr", "Price to Levered Free Cash Flow per Share"),
    33: ("TR.F.CAPEXTotPerShr", "Capital Expenditures - Total per Share"),
    34: ("TR.F.CAPEXtoDeprAndAmort", "Capital Expenditures to Depreciation and Amortization"),
    35: ("TR.F.CAPEXtoNetCashFlowOp", "Capital Expenditures to Net Cash Flow from Operations"),
    36: ("TR.F.NetIncPctofNetCashFlowOp", "Net Income Percentage of Net Cash Flow from Operations"),
    37: ("TR.F.DeprAndDeplPctofNetCashFlowOp", "Depreciation and Depletion Percentage of Net Cash Flow from Operations"),
    38: ("TR.FreeCashFlow", "Free Cash Flow"),
    39: ("TR.F.LeveredFOCFPerShr", "Levered Free Cash Flow per Share"),
    40: ("TR.F.NetDebtPoPAvg", "Net Debt Point of Period Average"),
    41: ("TR.F.DebtLTTotPctEV", "Long-Term Debt Total Percentage of Enterprise Value"),
    42: ("TR.F.NetDebtPctEV", "Net Debt Percentage of Enterprise Value"),
    43: ("TR.F.PriceToTangBVPerShr", "Price to Tangible Book Value per Share"),
    44: ("TR.F.ReturnAvgTotEquityPct5YrAvg", "Return on Average Total Equity - %, 5-Year Average"),
    45: ("TR.F.TotalLiabilitiesPopPct", "Total Liabilities - Point of Period Percentage"),
    46: ("TR.F.TotEquityPctofTotAssets", "Total Equity Percentage of Total Assets"),
    47: ("TR.F.PriceToNetCFOpPerShrTTM", "Price to Net Cash Flow from Operations per Share - TTM"),
    48: ("TR.F.PriceToFOCFPerShrIssue", "Price to Free Cash Flow per Share - Issue"),
    49: ("TR.F.PriceToLeveredFOCFPerShrIssue", "Price to Levered Free Cash Flow per Share - Issue"),
    50: ("TR.F.PPE", "Property, Plant & Equipment (Year over Year Average)"),
    51: ("TR.F.TotalFixedAssets", "Total Fixed Assets"),
    52: ("TR.F.NetOperatingAssets", "Net Operating Assets"),
    53: ("TR.IssuerRating", "Issuer Rating"),
    54: ("TR.F.TotRevPerShr", "Total Revenue per Share"),
    55: ("TR.F.PriceToCFPerShr", "Price to Cash Flow per Share"),
    56: ("TR.F.PriceToTotRevPerShr", "Price to Total Revenue per Share"),
    57: ("TR.F.AssetTurnover", "Asset Turnover"),
    58: ("TR.F.EBIT", "EBIT"),
    59: ("TR.TotalAssets", "Total Assets"),
    60: ("TR.TotalDebt", "Total Debt"),
    61: ("TR.F.EVToEBITDA", "Enterprise Value to EBITDA"),
    62: ("TR.F.EVToEBIT", "Enterprise Value to EBIT"),
    63: ("TR.GrossProfit", "Gross Profit"),
    64: ("TR.EBITDA", "EBITDA"),
    65: ("TR.EnterpriseValue", "Enterprise Value"),
    66: ("TR.Turnover", "Turnover"),
    67: ("TR.TotalLiabilities", "Total Liabilities"),
    70: ("TR.CashFlowFromOperationsMean", "Cash Flow From Operations - Mean"),
    71: ("TR.CashFlowFromOperationsActValue", "Cash Flow From Operations - Actual"),
    72: ("TR.F.NetCashFlowOP", "Net Cash Flow from Operating Activities"),
    73: ("TR.F.CF", "Cash Flow"),
    74: ("TR.F.NetIncAfterTax", "Net Income after Tax"),
    26: ("TR.F.WkgCapPerShr", "Working Capital per Share"),
    70: ("TR.F.NetIncAfterTax", "Net Income after Tax"),
    71: ("TR.F.NetIncAfterMinIntr", "Net Income after Minority Interest"),
    73: ("TR.NetProfitActValue", "Net Income - Actual"),
    76: ("TR.EPSActValue", "Earnings Per Share - Actual"),
    26: ("TR.F.WkgCap", "Working Capital"),
      2: ("TR.EPSMean", "Earnings Per Share - Mean"),
    3: ("TR.RepEPSMean", "Earnings Per Share Reported - Mean"),
    4: ("TR.DPSMean", "Dividend Per Share - Mean"),
    5: ("TR.DPSActValue", "Dividend Per Share - Actual"),
    8: ("TR.F.TotRevPerShr", "Total Revenue per Share"),
    16: ("TR.CFPSActValue", "Cash Flow Per Share - Actual"),
    17: ("TR.BVPSActValue", "Book Value Per Share - Actual"),
    17: ("TR.FCFPSActValue", "Free Cash Flow Per Share - Actual"),
    21: ("TR.EPSActValue", "Earnings Per Share - Actual"),
    24: ("TR.F.PriceToCFPerShr", "Price to Cash Flow per Share"),
    28: ("TR.F.NetDebtPerShr", "Net Debt per Share"),
    30: ("TR.F.TangBVPerShr", "Tangible Book Value per Share"),
    32: ("TR.F.PriceToNetCFOpPerShr", "Price to Cash Flow from Operations per Share"),
    35: ("TR.F.NetCFOpPerShr", "Cash Flow from Operations per Share"),
    36: ("TR.F.CAPEXTotPerShr", "Capital Expenditures - Total per Share"),
    46: ("TR.CFPSActValue", "Cash Flow Per Share - Actual"),
    48: ("TR.DPSActValue", "Dividend Per Share - Actual"),
    50: ("TR.BVPSActValue", "Book Value Per Share - Actual"),
    
}

available_fields = {
    # Income Statement
    1: ("ALL", "All Fields"),
    2: ("TR.Revenue", "Revenue"), 
    3: ("TR.F.ReturnAvgComEqPctTTM", "Return on Average Common Equity - %, TTM"),
    4: ("TR.F.ReturnAvgTotAssetsPctTTM", "Return on Average Total Assets - %, TTM"),
    5: ("TR.F.PriceToBookValuePerShrIssue", "Price to Book Value per Share - Issue"),
    6: ("TR.F.PriceToNetCFOpPerShr", "Price to Net Cash Flow from Operations per Share"),
    7: ("TR.F.PriceToFOCFPerShr", "Price to Free Cash Flow per Share"),
    8: ("TR.F.PriceToLeveredFOCFPerShr", "Price to Levered Free Cash Flow per Share"),
    9: ("TR.F.PriceToCFPerShr", "Price to Cash Flow per Share"),
    10: ("TR.F.PriceToTotRevPerShr", "Price to Total Revenue per Share"),
    11: ("TR.GrossProfit", "Gross Profit"),
    12: ("TR.EBITDA", "EBITDA"),
    13: ("TR.EBIT", "EBIT"),
    14: ("TR.F.PriceToTangBVPerShr", "Price to Tangible Book Value per Share"),
    15: ("TR.EPSActValue", "Earnings Per Share - Actual"),
    16: ("TR.DPSActValue", "Dividend Per Share - Actual"),
    17: ("TR.BVPSActValue", "Book Value Per Share - Actual"),
    74: ("TR.F.NetIncAfterTax", "Net Income after Tax"),
    75: ("TR.NetProfitActValue", "Net Income - Actual"),
    80: ("TR.FCFActValue","Free Cash Flow - Actual"),
    82: ("TR.F.UnLeveredFOCF","Unlevered Free Cash Flow"),

    # Balance Sheet
    18: ("TR.CompanyMarketCap", "Company Market Cap"),
    19: ("TR.TotalEquity", "Total Equity"),
    20: ("TR.F.NetPPEPctOfTotAssets", "Net Property, Plant & Equipment Percentage of Total Assets"),
    21: ("TR.F.GoodwNetPctOfTotAssets", "Goodwill - Net Percentage of Total Assets"),
    22: ("TR.F.IntangNetPctOfTotAssets", "Intangibles - Net Percentage of Total Assets"),
    23: ("TR.F.TotalFixedAssets", "Total Fixed Assets"),
    24: ("TR.F.NetOperatingAssets", "Net Operating Assets"),
    25: ("TR.TotalAssets", "Total Assets"),
    26: ("TR.TotalDebt", "Total Debt"),
    27: ("TR.F.DebtLTTotPctEV", "Long-Term Debt Total Percentage of Enterprise Value"),
    28: ("TR.F.NetDebtPctEV", "Net Debt Percentage of Enterprise Value"),
    29: ("TR.F.NetDebtPoPAvg", "Net Debt Point of Period Average"),
    30: ("TR.F.TotEquityPctofTotAssets", "Total Equity Percentage of Total Assets"),
    31: ("TR.F.WkgCapToTotAssets", "Working Capital to Total Assets"),
    32: ("TR.F.WkgCapPerShr", "Working Capital per Share"),
    33: ("TR.F.WkgCapNonCashPerShr", "Working Capital - Non-Cash per Share"),
    34: ("TR.F.TangBVPerShr", "Tangible Book Value per Share"),
    35: ("TR.F.CurrRatio", "Current Ratio"),  # Lagt til
    36: ("TR.F.QuickRatio", "Quick Ratio"),   # Lagt til

    # Cash Flow
    37: ("TR.F.CAPEXTotPerShr", "Capital Expenditures - Total per Share"),
    38: ("TR.F.CAPEXtoDeprAndAmort", "Capital Expenditures to Depreciation and Amortization"),
    39: ("TR.F.CAPEXtoNetCashFlowOp", "Capital Expenditures to Net Cash Flow from Operations"),
    40: ("TR.FreeCashFlow", "Free Cash Flow"),
    41: ("TR.F.LeveredFOCFPerShr", "Levered Free Cash Flow per Share"),
    42: ("TR.CashFlowFromOperationsActValue", "Cash Flow From Operations - Actual"),
    43: ("TR.F.NetCashFlowOP", "Net Cash Flow from Operating Activities"),
    44: ("TR.F.NetCFOpPerShr", "Cash Flow from Operations per Share"),
    45: ("TR.F.DeprAndDeplPctofNetCashFlowOp", "Depreciation and Depletion Percentage of Net Cash Flow from Operations"),
    46: ("TR.F.NetIncPctofNetCashFlowOp", "Net Income Percentage of Net Cash Flow from Operations"),
    47: ("TR.F.PriceToNetCFOpPerShrTTM", "Price to Net Cash Flow from Operations per Share - TTM"),

    # Other
    48: ("TR.WACCBeta", "Beta"),
    49: ("TR.BIDPRICE", "Bid Price"),
    50: ("TR.ASKPRICE", "Ask Price"),
    51: ("TR.IssuerRating", "Issuer Rating"),
    52: ("TR.F.ReturnAvgTotEquityPct5YrAvg", "Return on Average Total Equity - %, 5-Year Average"),
    53: ("TR.F.TotalLiabilitiesPopPct", "Total Liabilities - Point of Period Percentage"),
    54: ("TR.F.AssetTurnover", "Asset Turnover"),
    55: ("TR.F.EVToEBITDA", "Enterprise Value to EBITDA"),
    56: ("TR.F.EVToEBIT", "Enterprise Value to EBIT"),
    57: ("TR.F.EV", "Enterprise Value"),    
    58: ("TR.Turnover", "Turnover"),
    59: ("TR.TotalLiabilities", "Total Liabilities"),
    60: ("TR.FreeFloatPct", "Free Float (Percent)"),
}


In [18]:
"""
##Fungerer:
available_fields = {
    1: ("ALL", "All Fields"),
    2: ("TR.CompanyMarketCap", "Company Market Cap"),
    3: ("TR.Revenue", "Revenue"),
    4: ("TR.TotalEquity", "Total Equity"),
    5: ("TR.F.ReturnAvgComEqPctTTM", "Return on Average Common Equity - %, TTM"),
    6: ("TR.F.ReturnAvgTotAssetsPctTTM", "Return on Average Total Assets - %, TTM"),
    7: ("TR.F.PriceToEPSDilutedExclExordNormTTM", "Price to EPS Diluted excl Exord - Norm - Total, TTM"),
    8: ("TR.F.PriceToEPSDilutedExclExordCommonIssue", "Price to EPS Diluted excl Exord, Common - Issue"),
    9: ("TR.F.PriceToEPSDilutedExclExordCommonIssue3YrAvg", "Price to EPS Diluted excl Exord, Common - Issue, 3 Yr Avg"),
    10: ("TR.F.PriceToEPSDilutedExclExordCommonIssue5YrAvg", "Price to EPS Diluted excl Exord, Common - Issue, 5 Yr Avg"),
    11: ("TR.F.PriceToEPSDilutedInclExordCommonIssue3YrAvg", "Price to EPS Diluted incl Exord, Common - Issue, 3 Yr Avg"),
    12: ("TR.F.PriceToEPSDilutedInclExordCommonIssue5YrAvg", "Price to EPS Diluted incl Exord, Common - Issue, 5 Yr Avg"),
    13: ("TR.F.PriceToHeadlineEPS", "Price to Headline EPS"),
    14: ("TR.F.PriceToHeadlineEPS3YrAvg", "Price to Headline EPS, 3 Yr Avg"),
    15: ("TR.F.PriceToHeadlineEPS5YrAvg", "Price to Headline EPS, 5 Yr Avg"),
    16: ("TR.F.PriceToHeadlineEPSTTM", "Price to Headline EPS, TTM"),
    17: ("TR.F.PriceToDPSCommonGrossIssue", "Price to DPS - Common - Gross - Issue"),
    18: ("TR.F.PriceToDPSCommonGrossIssueTTM", "Price to DPS - Common - Gross - Issue, TTM"),
    19: ("TR.F.PriceToBookValuePerShrIssue", "Price to Book Value per Share - Issue"),
    20: ("TR.F.PriceToBookValuePerShrIssue3YrAvg", "Price to Book Value per Share - Issue, 3 Yr Avg"),
    21: ("TR.F.PriceToBookValuePerShrIssue5YrAvg", "Price to Book Value per Share - Issue, 5 Yr Avg"),
    22: ("TR.F.PriceToBookValuePerShr3YrAvg", "Price to Book Value per Share, 3 Yr Avg"),
    23: ("TR.F.PriceToBookValuePerShr5YrAvg", "Price to Book Value per Share, 5 Yr Avg"),
    24: ("TR.F.PriceToTangibleBookValuePerShrIssue", "Price to Tangible Book Value per Share - Issue"),
    25: ("TR.F.PriceToTangibleBookValuePerShrIssue3YrAvg", "Price to Tangible Book Value per Share - Issue, 3 Yr Avg"),
    26: ("TR.F.PriceToTangibleBookValuePerShrIssue5YrAvg", "Price to Tangible Book Value per Share - Issue, 5 Yr Avg"),
    27: ("TR.F.PriceToTangibleBookValuePerShr3YrAvg", "Price to Tangible Book Value per Share, 3 Yr Avg"),
    28: ("TR.F.PriceToTangibleBookValuePerShr5YrAvg", "Price to Tangible Book Value per Share, 5 Yr Avg"),
    29: ("TR.F.EPSBasicInclExordCommonIssue5YrAvg", "EPS Basic incl Exord, Common - Issue, 5 Yr Avg"),
    30: ("TR.F.EPSBasicFromDiscontinuedOpsExord", "EPS Basic from Discontinued Operations & Exord"),
    31: ("TR.F.EPSBasicFromDiscontinuedOpsExordPoPDiff", "EPS Basic from Discontinued Operations & Exord, PoP Diff"),
    32: ("TR.F.EPSDilutedInclExordCommonIssue3YrAvg", "EPS Diluted incl Exord, Common - Issue, 3 Yr Avg"),
    33: ("TR.F.EPSDilutedInclExordCommonIssue5YrAvg", "EPS Diluted incl Exord, Common - Issue, 5 Yr Avg"),
    34: ("TR.F.MarketCap", "Market Capitalization"),
    35: ("TR.F.NetCashYield", "Net Cash Yield - %"),
    36: ("TR.F.PriceCloseEndOfPeriodIssue3YrAvg", "Price Close (End of Period) - Issue, 3 Yr Avg"),
    37: ("TR.F.PriceCloseEndOfPeriodIssue5YrAvg", "Price Close (End of Period) - Issue, 5 Yr Avg"),
    38: ("TR.F.PriceCloseEndOfPeriod3YrAvg", "Price Close (End of Period), 3 Yr Avg"),
    39: ("TR.F.PriceCloseEndOfPeriod5YrAvg", "Price Close (End of Period), 5 Yr Avg"),
    40: ("TR.F.WorkingCapitalNonCashPerShrIssueToPricePct", "Working Capital - Non-Cash per Share - Issue to Price - %"),
    41: ("TR.F.WorkingCapitalPerShrIssueToPricePct", "Working Capital per Share - Issue to Price - %"),
    42: ("TR.F.CurrentEnterpriseValue", "Curr Enterprise Value"),
    43: ("TR.F.PriceCloseEndOfPeriod", "Price Close (End of Period)"),
    44: ("TR.F.MarketCapOS", "Market Capitalization (O/S)"),
    45: ("TR.F.CurrentEnterpriseValueOS", "Curr Enterprise Value (O/S)"),
    46: ("TR.F.CurrentPriceHighToBookValuePerShr", "Curr Price High to Book Value per Share"),
    47: ("TR.F.CurrentPriceHighToBookValuePerShrIssue", "Curr Price High to Book Value per Share - Issue"),
    48: ("TR.F.CurrentPriceLowToBookValuePerShr", "Curr Price Low to Book Value per Share"),
    49: ("TR.F.CurrentPriceLowToBookValuePerShrIssue", "Curr Price Low to Book Value per Share - Issue"),
    50: ("TR.F.FreeCashFlowYieldTTM", "Free Cash Flow Yield - %, TTM"),
    51: ("TR.F.FreeCashFlowYield3YrAvg", "Free Cash Flow Yield - %, 3 Yr Avg"),
    52: ("TR.F.FreeCashFlowYield5YrAvg", "Free Cash Flow Yield - %, 5 Yr Avg"),
}

# Fields that does not work
Fields That Do Not Work
Return on Average Total Equity - %, 5 Yr Avg
Price to Cash Flow per Share - Issue
Price to Cash Flow per Share - Issue, TTM
Price to EPS Diluted excl Exord, Com - Issue, 5 Yr High
Price to EPS Diluted excl Exord, Com - Issue, 5 Yr Low
Price to DPS - Common - Net - Issue
Price to DPS - Common - Net - Issue, TTM
Price to Cash Flow per Share, 3 Yr Avg
Price to Cash Flow per Share, 5 Yr Avg
Price to FCF Net of Dividends per Share - Issue
Price to FCF Net of Dividends per Share - Issue, TTM
Price to FCF Net of Dividends per Share - Issue, 3 Yr Avg
Price to FCF Net of Dividends per Share - Issue, 5 Yr Avg
Price to FCF per Share - Issue
Price to FCF per Share - Issue, TTM
EPS Basic from Discontinued Operations & Exord, PoP % Chg
"""

'\n##Fungerer:\navailable_fields = {\n    1: ("ALL", "All Fields"),\n    2: ("TR.CompanyMarketCap", "Company Market Cap"),\n    3: ("TR.Revenue", "Revenue"),\n    4: ("TR.TotalEquity", "Total Equity"),\n    5: ("TR.F.ReturnAvgComEqPctTTM", "Return on Average Common Equity - %, TTM"),\n    6: ("TR.F.ReturnAvgTotAssetsPctTTM", "Return on Average Total Assets - %, TTM"),\n    7: ("TR.F.PriceToEPSDilutedExclExordNormTTM", "Price to EPS Diluted excl Exord - Norm - Total, TTM"),\n    8: ("TR.F.PriceToEPSDilutedExclExordCommonIssue", "Price to EPS Diluted excl Exord, Common - Issue"),\n    9: ("TR.F.PriceToEPSDilutedExclExordCommonIssue3YrAvg", "Price to EPS Diluted excl Exord, Common - Issue, 3 Yr Avg"),\n    10: ("TR.F.PriceToEPSDilutedExclExordCommonIssue5YrAvg", "Price to EPS Diluted excl Exord, Common - Issue, 5 Yr Avg"),\n    11: ("TR.F.PriceToEPSDilutedInclExordCommonIssue3YrAvg", "Price to EPS Diluted incl Exord, Common - Issue, 3 Yr Avg"),\n    12: ("TR.F.PriceToEPSDilutedInclExordCo

## Fields og velge fra

In [122]:
available_fields = {
    1: ("ALL", "All Fields"),
    2: ("TR.CompanyMarketCap", "Company Market Cap"),
    3: ("TR.Revenue", "Revenue"),
    4: ("TR.TotalEquity", "Total Equity"),
    5: ("TR.F.ReturnAvgComEqPctTTM", "Return on Average Common Equity - %, TTM"),
    6: ("TR.F.ReturnAvgTotAssetsPctTTM", "Return on Average Total Assets - %, TTM"),
    7: ("TR.F.PriceToBookValuePerShrIssue", "Price to Book Value per Share - Issue"),
    8: ("TR.WACCBeta", "Beta"),
    9: ("TR.F.CashCashEquivTotPctOfTotAssets", "Cash & Cash Equivalents - Total Percentage of Total Assets"),
    10: ("TR.F.TotCashSTInvstPctOfTotAssets", "Total Cash & ST Investments Percentage of Total Assets"),
    11: ("TR.F.RcvblPctOfTotAssets", "Receivables Percentage of Total Assets"),
    12: ("TR.F.InvntPctOfTotAssets", "Inventory Percentage of Total Assets"),
    13: ("TR.BIDPRICE", "Bid Price"),
    14: ("TR.F.QuickRatio", "Quick Ratio"),
    15: ("TR.ASKPRICE", "Ask Price"),
    16: ("TR.F.PriceToEBITDAPerShr", "Price to EBITDA per Share"),
    17: ("TR.BVPSActValue", "Book Value Per Share - Actual"),
    19: ("TR.F.WkgCapToTotAssets", "Working Capital to Total Assets"),
    21: ("TR.F.NetPPEPctOfTotAssets", "Net Property, Plant & Equipment Percentage of Total Assets"),
    22: ("TR.F.GoodwNetPctOfTotAssets", "Goodwill - Net Percentage of Total Assets"),
    23: ("TR.F.IntangNetPctOfTotAssets", "Intangibles - Net Percentage of Total Assets"),
    24: ("TR.F.PriceToCFPerShr", "Price to Cash Flow per Share"),
    25: ("TR.F.WkgCapNonCashPerShr", "Working Capital - Non-Cash per Share"),
    26: ("TR.F.WkgCapPerShr", "Working Capital per Share"),
    28: ("TR.F.WkgCap5YrAvg", "Working Capital 5-Year Average"),
    30: ("TR.F.PriceToNetCFOpPerShr", "Price to Net Cash Flow from Operations per Share"),
    31: ("TR.F.PriceToFOCFPerShr", "Price to Free Cash Flow per Share"),
    32: ("TR.F.PriceToNetCFOpPerShr", "Price to Cash Flow from Operations per Share"),
    33: ("TR.F.CAPEXTotPerShr", "Capital Expenditures - Total per Share"),
    34: ("TR.F.CAPEXtoDeprAndAmort", "Capital Expenditures to Depreciation and Amortization"),
    35: ("TR.F.NetCFOpPerShr", "Cash Flow from Operations per Share"),
    36: ("TR.F.CAPEXTotPerShr", "Capital Expenditures - Total per Share"),
    37: ("TR.F.DeprAndDeplPctofNetCashFlowOp", "Depreciation and Depletion Percentage of Net Cash Flow from Operations"),
    38: ("TR.FreeCashFlow", "Free Cash Flow"),
    39: ("TR.F.LeveredFOCFPerShr", "Levered Free Cash Flow per Share"),
    40: ("TR.F.NetDebtPoPAvg", "Net Debt Point of Period Average"),
    41: ("TR.F.DebtLTTotPctEV", "Long-Term Debt Total Percentage of Enterprise Value"),
    42: ("TR.F.NetDebtPctEV", "Net Debt Percentage of Enterprise Value"),
    43: ("TR.F.PriceToTangBVPerShr", "Price to Tangible Book Value per Share"),
    44: ("TR.F.ReturnAvgTotEquityPct5YrAvg", "Return on Average Total Equity - %, 5-Year Average"),
    45: ("TR.F.TotalLiabilitiesPopPct", "Total Liabilities - Point of Period Percentage"),
    46: ("TR.CFPSActValue", "Cash Flow Per Share - Actual"),
    47: ("TR.F.PriceToNetCFOpPerShrTTM", "Price to Net Cash Flow from Operations per Share - TTM"),
    48: ("TR.DPSActValue", "Dividend Per Share - Actual"),
    49: ("TR.F.PriceToLeveredFOCFPerShrIssue", "Price to Levered Free Cash Flow per Share - Issue"),
    50: ("TR.F.PPE", "Property, Plant & Equipment (Year over Year Average)"),
    51: ("TR.F.TotalFixedAssets", "Total Fixed Assets"),
    52: ("TR.F.NetOperatingAssets", "Net Operating Assets"),
    53: ("TR.IssuerRating", "Issuer Rating"),
    54: ("TR.F.TotRevPerShr", "Total Revenue per Share"),
    55: ("TR.F.PriceToCFPerShr", "Price to Cash Flow per Share"),
    56: ("TR.F.PriceToTotRevPerShr", "Price to Total Revenue per Share"),
    57: ("TR.F.AssetTurnover", "Asset Turnover"),
    58: ("TR.F.EBIT", "EBIT"),
    59: ("TR.TotalAssets", "Total Assets"),
    60: ("TR.TotalDebt", "Total Debt"),
    61: ("TR.F.EVToEBITDA", "Enterprise Value to EBITDA"),
    62: ("TR.F.EVToEBIT", "Enterprise Value to EBIT"),
    63: ("TR.GrossProfit", "Gross Profit"),
    64: ("TR.EBITDA", "EBITDA"),
    65: ("TR.EnterpriseValue", "Enterprise Value"),
    66: ("TR.Turnover", "Turnover"),
    67: ("TR.TotalLiabilities", "Total Liabilities"),
    70: ("TR.CashFlowFromOperationsMean", "Cash Flow From Operations - Mean"),
    71: ("TR.F.NetIncAfterMinIntr", "Net Income after Minority Interest"),
    72: ("TR.F.NetCashFlowOP", "Net Cash Flow from Operating Activities"),
    73: ("TR.NetProfitActValue", "Net Income - Actual"),
    74: ("TR.F.NetIncAfterTax", "Net Income after Tax"),
    76: ("TR.EPSActValue", "Earnings Per Share - Actual"),
}



## Felles funksjoner

In [1]:
import refinitiv.data as rd
import pandas as pd
from datetime import datetime
import os

# Set up the App Key as a shared variable
APP_KEY = "74859fbc21ff4d5aa1aa2ce565b1a8c8c0f4d37a"

def open_refinitiv_session():
    """Opens a Refinitiv session with the Refinitiv Data Platform."""
    rd.open_session(app_key=APP_KEY)
    print("Connection successful! You are now connected to the Refinitiv Data Platform.")

# Available fields with TR codes


available_fields = {
    # Income Statement
    1: ("ALL", "All Fields"),
    2: ("TR.Revenue", "Revenue"), 
    3: ("TR.F.ReturnAvgComEqPctTTM", "Return on Average Common Equity - %, TTM"),
    4: ("TR.F.ReturnAvgTotAssetsPctTTM", "Return on Average Total Assets - %, TTM"),
    5: ("TR.F.PriceToBookValuePerShrIssue", "Price to Book Value per Share - Issue"),
    6: ("TR.F.PriceToNetCFOpPerShr", "Price to Net Cash Flow from Operations per Share"),
    7: ("TR.F.PriceToFOCFPerShr", "Price to Free Cash Flow per Share"),
    8: ("TR.F.PriceToLeveredFOCFPerShr", "Price to Levered Free Cash Flow per Share"),
    9: ("TR.F.PriceToCFPerShr", "Price to Cash Flow per Share"),
    10: ("TR.F.PriceToTotRevPerShr", "Price to Total Revenue per Share"),
    11: ("TR.GrossProfit", "Gross Profit"),
    12: ("TR.EBITDA", "EBITDA"),
    13: ("TR.EBIT", "EBIT"),
    14: ("TR.F.PriceToTangBVPerShr", "Price to Tangible Book Value per Share"),
    15: ("TR.EPSActValue", "Earnings Per Share - Actual"),
    16: ("TR.DPSActValue", "Dividend Per Share - Actual"),
    17: ("TR.BVPSActValue", "Book Value Per Share - Actual"),
    74: ("TR.F.NetIncAfterTax", "Net Income after Tax"),
    75: ("TR.NetProfitActValue", "Net Income - Actual"),
    80: ("TR.FCFActValue","Free Cash Flow - Actual"),
    82: ("TR.F.UnLeveredFOCF","Unlevered Free Cash Flow"),

    # Balance Sheet
    18: ("TR.CompanyMarketCap", "Company Market Cap"),
    19: ("TR.TotalEquity", "Total Equity"),
    20: ("TR.F.NetPPEPctOfTotAssets", "Net Property, Plant & Equipment Percentage of Total Assets"),
    21: ("TR.F.GoodwNetPctOfTotAssets", "Goodwill - Net Percentage of Total Assets"),
    22: ("TR.F.IntangNetPctOfTotAssets", "Intangibles - Net Percentage of Total Assets"),
    23: ("TR.F.TotalFixedAssets", "Total Fixed Assets"),
    24: ("TR.F.NetOperatingAssets", "Net Operating Assets"),
    25: ("TR.TotalAssets", "Total Assets"),
    26: ("TR.TotalDebt", "Total Debt"),
    27: ("TR.F.DebtLTTotPctEV", "Long-Term Debt Total Percentage of Enterprise Value"),
    28: ("TR.F.NetDebtPctEV", "Net Debt Percentage of Enterprise Value"),
    29: ("TR.F.NetDebtPoPAvg", "Net Debt Point of Period Average"),
    30: ("TR.F.TotEquityPctofTotAssets", "Total Equity Percentage of Total Assets"),
    31: ("TR.F.WkgCapToTotAssets", "Working Capital to Total Assets"),
    32: ("TR.F.WkgCapPerShr", "Working Capital per Share"),
    33: ("TR.F.WkgCapNonCashPerShr", "Working Capital - Non-Cash per Share"),
    34: ("TR.F.TangBVPerShr", "Tangible Book Value per Share"),
    35: ("TR.F.CurrRatio", "Current Ratio"),  # Lagt til
    36: ("TR.F.QuickRatio", "Quick Ratio"),   # Lagt til

    # Cash Flow
    37: ("TR.F.CAPEXTotPerShr", "Capital Expenditures - Total per Share"),
    38: ("TR.F.CAPEXtoDeprAndAmort", "Capital Expenditures to Depreciation and Amortization"),
    39: ("TR.F.CAPEXtoNetCashFlowOp", "Capital Expenditures to Net Cash Flow from Operations"),
    40: ("TR.FreeCashFlow", "Free Cash Flow"),
    41: ("TR.F.LeveredFOCFPerShr", "Levered Free Cash Flow per Share"),
    42: ("TR.CashFlowFromOperationsActValue", "Cash Flow From Operations - Actual"),
    43: ("TR.F.NetCashFlowOP", "Net Cash Flow from Operating Activities"),
    44: ("TR.F.NetCFOpPerShr", "Cash Flow from Operations per Share"),
    45: ("TR.F.DeprAndDeplPctofNetCashFlowOp", "Depreciation and Depletion Percentage of Net Cash Flow from Operations"),
    46: ("TR.F.NetIncPctofNetCashFlowOp", "Net Income Percentage of Net Cash Flow from Operations"),
    47: ("TR.F.PriceToNetCFOpPerShrTTM", "Price to Net Cash Flow from Operations per Share - TTM"),

    # Other
    48: ("TR.WACCBeta", "Beta"),
    49: ("TR.BIDPRICE", "Bid Price"),
    50: ("TR.ASKPRICE", "Ask Price"),
    51: ("TR.IssuerRating", "Issuer Rating"),
    52: ("TR.F.ReturnAvgTotEquityPct5YrAvg", "Return on Average Total Equity - %, 5-Year Average"),
    53: ("TR.F.TotalLiabilitiesPopPct", "Total Liabilities - Point of Period Percentage"),
    54: ("TR.F.AssetTurnover", "Asset Turnover"),
    55: ("TR.F.EVToEBITDA", "Enterprise Value to EBITDA"),
    56: ("TR.F.EVToEBIT", "Enterprise Value to EBIT"),
    57: ("TR.F.EV", "Enterprise Value"),
    59: ("TR.Turnover", "Turnover"),
    60: ("TR.TotalLiabilities", "Total Liabilities"),
    61: ("TR.FreeFloatPct", "Free Float (Percent)"),
}





def fetch_data(tickers, selected_field, expected_column, currency):
    """Fetches data from Refinitiv based on tickers, field, and expected column."""
    data = pd.DataFrame()  # Collects data for all companies

    for ticker in tickers:
        try:
            response = rd.content.fundamental_and_reference.Definition(
                universe=[ticker],
                fields=[selected_field],
                parameters={"Scale": 6, "SDate": "1990-01-01", "EDate": 0, "FRQ": "FY", "Curn": currency}
            ).get_data()

            if response and not response.data.df.empty:
                df = response.data.df

                if expected_column not in df.columns:
                    print(f"Column '{expected_column}' does not exist in the response for {ticker}. Skipping.")
                    continue

                company_df = df[[expected_column]].copy()
                company_df.columns = [ticker]

                available_dates = df.get("Date")
                if available_dates is not None:
                    company_df["Date"] = pd.to_datetime(available_dates)
                else:
                    num_rows = company_df.shape[0]
                    current_year = datetime.now().year
                    dates = pd.date_range(end=f"{current_year-2}-12-31", periods=num_rows, freq="YE")
                    company_df["Date"] = dates

                company_df = company_df[company_df["Date"] <= pd.to_datetime(f"{datetime.now().year}-12-31")]
                company_df.set_index("Date", inplace=True)

                data = pd.concat([data, company_df], axis=1)
                print(f"Data retrieved for {ticker}")
            else:
                print(f"No data available for {ticker}")

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

def fetch_multiple_fields(tickers, selected_fields, currency):
    """
    Fetches data for multiple fields based on the user's selection.
    :param tickers: List of tickers.
    :param selected_fields: List of selected fields (tuples with field and expected column name).
    :param currency: Currency for the data retrieval.
    :return: Dictionary with data for each field.
    """
    combined_data = {}
    for selected_field, expected_column in selected_fields:
        print(f"Fetching data for field: {expected_column}")
        data = fetch_data(tickers, selected_field, expected_column, currency)
        combined_data[expected_column] = data
    return combined_data

def save_data_to_excel(retrieved_data, file_name, sheet_prefix):
    """
    Saves data to an Excel file with a specified file name and sheet prefix.
    Ensures all resources are properly closed after writing.
    """
    if not retrieved_data:
        print("No data available to save.")
        return

    folder_name = os.path.dirname(file_name)
    if folder_name and not os.path.exists(folder_name):
        os.makedirs(folder_name, exist_ok=True)

    try:
        with pd.ExcelWriter(file_name, mode="w", engine="openpyxl") as writer:
            has_data = False  # Track if any sheet contains data

            for field, data in retrieved_data.items():
                if not data.empty:
                    # Shorten sheet name if necessary
                    sheet_name = f"{sheet_prefix} - {field}"[:30]
                    data.to_excel(writer, sheet_name=sheet_name)
                    print(f"Data saved in '{file_name}', Sheet: {sheet_name}.")
                    has_data = True
                else:
                    print(f"No data available for '{field}'. Skipping.")

            if not has_data:
                # Add an empty sheet if no data was written to any sheet
                empty_df = pd.DataFrame({"Message": ["No data available"]})
                empty_df.to_excel(writer, sheet_name="No Data")
                print(f"No data available. An empty sheet was created in '{file_name}'.")

    except Exception as e:
        print(f"Error while saving to Excel: {e}")

    finally:
        # Explicitly close the writer to ensure all resources are released
        try:
            writer.close()
            print("Excel writer closed.")
        except Exception as close_error:
            print(f"Error closing Excel writer: {close_error}")

    print(f"Excel file '{file_name}' should now be fully closed and accessible.")


def close_refinitiv_session():
    """Closes the Refinitiv session."""
    rd.close_session()
    print("Session closed.")

# Fix Pandas FutureWarning
pd.set_option('future.no_silent_downcasting', True)

In [1]:
import refinitiv.data as rd
import pandas as pd
from datetime import datetime
import os

# Set up the App Key as a shared variable
APP_KEY = "74859fbc21ff4d5aa1aa2ce565b1a8c8c0f4d37a"

def open_refinitiv_session():
    """Opens a Refinitiv session with the Refinitiv Data Platform."""
    rd.open_session(app_key=APP_KEY)
    print("Connection successful! You are now connected to the Refinitiv Data Platform.")

# Oppdaterte felter
available_fields = {
    1: ("ALL", "All Fields"),
    49: ("TR.BIDPRICE", "Bid Price"),
    50: ("TR.ASKPRICE", "Ask Price"),
    59: ("TR.Turnover", "Turnover"),
    60: ("TR.VOLUME", "Volume"),
    61: ("TR.FreeFloatPct", "Free Float (Percent)"),
    62: ("TR.FiPrice", "Price"),
    63: ("TR.CLOSEPRICE", "Close Price"),
    64: ("TR.Price52WeekHigh", "Price 52 Week High"),
    65: ("TR.Price52WeekLow", "Price 52 Week Low")
}

def fetch_data(tickers, selected_field, expected_column, currency):
    """Fetches data from Refinitiv based on tickers, field, and expected column."""
    data = pd.DataFrame()  # Collects data for all companies

    for ticker in tickers:
        try:
            response = rd.content.fundamental_and_reference.Definition(
                universe=[ticker],
                fields=[selected_field],
                parameters={"Scale": 6, "SDate": "1990-01-01", "EDate": 0, "FRQ": "D", "Curn": currency}
            ).get_data()

            if response and not response.data.df.empty:
                df = response.data.df

                if expected_column not in df.columns:
                    print(f"Column '{expected_column}' does not exist in the response for {ticker}. Skipping.")
                    continue

                company_df = df[[expected_column]].copy()
                company_df.columns = [ticker]

                available_dates = df.get("Date")
                if available_dates is not None:
                    company_df["Date"] = pd.to_datetime(available_dates)
                else:
                    num_rows = company_df.shape[0]
                    current_year = datetime.now().year
                    dates = pd.date_range(end=f"{current_year-2}-12-31", periods=num_rows, freq="D")
                    company_df["Date"] = dates

                company_df = company_df[company_df["Date"] <= pd.to_datetime(f"{datetime.now().year}-12-31")]
                company_df.set_index("Date", inplace=True)

                data = pd.concat([data, company_df], axis=1)
                print(f"Data retrieved for {ticker}")
            else:
                print(f"No data available for {ticker}")

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

def fetch_multiple_fields(tickers, selected_fields, currency):
    """
    Fetches data for multiple fields based on the user's selection.
    :param tickers: List of tickers.
    :param selected_fields: List of selected fields (tuples with field and expected column name).
    :param currency: Currency for the data retrieval.
    :return: Dictionary with data for each field.
    """
    combined_data = {}
    for selected_field, expected_column in selected_fields:
        print(f"Fetching data for field: {expected_column}")
        data = fetch_data(tickers, selected_field, expected_column, currency)
        combined_data[expected_column] = data
    return combined_data

def save_data_to_excel(retrieved_data, file_name, sheet_prefix):
    """
    Saves data to an Excel file with a specified file name and sheet prefix.
    Ensures all resources are properly closed after writing.
    """
    if not retrieved_data:
        print("No data available to save.")
        return

    folder_name = os.path.dirname(file_name)
    if folder_name and not os.path.exists(folder_name):
        os.makedirs(folder_name, exist_ok=True)

    try:
        with pd.ExcelWriter(file_name, mode="w", engine="openpyxl") as writer:
            has_data = False  # Track if any sheet contains data

            for field, data in retrieved_data.items():
                if not data.empty:
                    # Shorten sheet name if necessary
                    sheet_name = f"{sheet_prefix} - {field}"[:30]
                    data.to_excel(writer, sheet_name=sheet_name)
                    print(f"Data saved in '{file_name}', Sheet: {sheet_name}.")
                    has_data = True
                else:
                    print(f"No data available for '{field}'. Skipping.")

            if not has_data:
                # Add an empty sheet if no data was written to any sheet
                empty_df = pd.DataFrame({"Message": ["No data available"]})
                empty_df.to_excel(writer, sheet_name="No Data")
                print(f"No data available. An empty sheet was created in '{file_name}'.")

    except Exception as e:
        print(f"Error while saving to Excel: {e}")

    finally:
        # Explicitly close the writer to ensure all resources are released
        try:
            writer.close()
            print("Excel writer closed.")
        except Exception as close_error:
            print(f"Error closing Excel writer: {close_error}")

    print(f"Excel file '{file_name}' should now be fully closed and accessible.")


def close_refinitiv_session():
    """Closes the Refinitiv session."""
    rd.close_session()
    print("Session closed.")

# Fix Pandas FutureWarning
pd.set_option('future.no_silent_downcasting', True)

In [2]:
import refinitiv.data as rd
import pandas as pd
from datetime import datetime
import os

# Set up the App Key as a shared variable
APP_KEY = "74859fbc21ff4d5aa1aa2ce565b1a8c8c0f4d37a"

def open_refinitiv_session():
    """Opens a Refinitiv session with the Refinitiv Data Platform."""
    rd.open_session(app_key=APP_KEY)
    print("Connection successful! You are now connected to the Refinitiv Data Platform.")

# Oppdaterte felter
available_fields = {
    1: ("ALL", "All Fields"),
    49: ("TR.FiPrice", "Price"),
    50: ("TR.CLOSEPRICE", "Close Price"),
    59: ("TR.Price52WeekHigh", "Price 52 Week High"),
    60: ("TR.Price52WeekLow", "Price 52 Week Low"),
    61: ("TR.FreeFloatPct", "Free Float (Percent)")
}

def fetch_data(tickers, selected_field, expected_column, currency):
    """Fetches data from Refinitiv based on tickers, field, and expected column."""
    data = pd.DataFrame()  # Collects data for all companies

    for ticker in tickers:
        try:
            response = rd.content.fundamental_and_reference.Definition(
                universe=[ticker],
                fields=[selected_field],
                parameters={"Scale": 6, "SDate": "1990-01-01", "EDate": 0, "FRQ": "M", "Curn": currency}
            ).get_data()

            if response and not response.data.df.empty:
                df = response.data.df

                if expected_column not in df.columns:
                    print(f"Column '{expected_column}' does not exist in the response for {ticker}. Skipping.")
                    continue

                company_df = df[[expected_column]].copy()
                company_df.columns = [ticker]

                available_dates = df.get("Date")
                if available_dates is not None:
                    company_df["Date"] = pd.to_datetime(available_dates)
                else:
                    num_rows = company_df.shape[0]
                    current_year = datetime.now().year
                    dates = pd.date_range(end=f"{current_year-2}-12-31", periods=num_rows, freq="M")
                    company_df["Date"] = dates

                company_df = company_df[company_df["Date"] <= pd.to_datetime(f"{datetime.now().year}-12-31")]
                company_df.set_index("Date", inplace=True)

                data = pd.concat([data, company_df], axis=1)
                print(f"Data retrieved for {ticker}")
            else:
                print(f"No data available for {ticker}")

        except Exception as e:
            print(f"Error fetching data for {ticker}: {e}")

    return data

def fetch_multiple_fields(tickers, selected_fields, currency):
    """
    Fetches data for multiple fields based on the user's selection.
    :param tickers: List of tickers.
    :param selected_fields: List of selected fields (tuples with field and expected column name).
    :param currency: Currency for the data retrieval.
    :return: Dictionary with data for each field.
    """
    combined_data = {}
    for selected_field, expected_column in selected_fields:
        print(f"Fetching data for field: {expected_column}")
        data = fetch_data(tickers, selected_field, expected_column, currency)
        combined_data[expected_column] = data
    return combined_data

def save_data_to_excel(retrieved_data, file_name, sheet_prefix):
    """
    Saves data to an Excel file with a specified file name and sheet prefix.
    Ensures all resources are properly closed after writing.
    """
    if not retrieved_data:
        print("No data available to save.")
        return

    folder_name = os.path.dirname(file_name)
    if folder_name and not os.path.exists(folder_name):
        os.makedirs(folder_name, exist_ok=True)

    try:
        with pd.ExcelWriter(file_name, mode="w", engine="openpyxl") as writer:
            has_data = False  # Track if any sheet contains data

            for field, data in retrieved_data.items():
                if not data.empty:
                    # Shorten sheet name if necessary
                    sheet_name = f"{sheet_prefix} - {field}"[:30]
                    data.to_excel(writer, sheet_name=sheet_name)
                    print(f"Data saved in '{file_name}', Sheet: {sheet_name}.")
                    has_data = True
                else:
                    print(f"No data available for '{field}'. Skipping.")

            if not has_data:
                # Add an empty sheet if no data was written to any sheet
                empty_df = pd.DataFrame({"Message": ["No data available"]})
                empty_df.to_excel(writer, sheet_name="No Data")
                print(f"No data available. An empty sheet was created in '{file_name}'.")

    except Exception as e:
        print(f"Error while saving to Excel: {e}")

    finally:
        # Explicitly close the writer to ensure all resources are released
        try:
            writer.close()
            print("Excel writer closed.")
        except Exception as close_error:
            print(f"Error closing Excel writer: {close_error}")

    print(f"Excel file '{file_name}' should now be fully closed and accessible.")


def close_refinitiv_session():
    """Closes the Refinitiv session."""
    rd.close_session()
    print("Session closed.")

# Fix Pandas FutureWarning
pd.set_option('future.no_silent_downcasting', True)

## Test

In [11]:
# Open Refinitiv session
open_refinitiv_session()

# Ask the user what they want to fetch
print("What do you want to fetch?")
for key, value in available_fields.items():
    print(f"{key}: {value[1]}")

try:
    selection = int(input("Select an option: "))
    if selection not in available_fields:
        print("Invalid selection. Program will exit.")
        close_refinitiv_session()
        exit()

    selected_field, expected_column = available_fields[selection]
except Exception as e:
    print(f"Error in selection: {e}. Program will exit.")
    close_refinitiv_session()
    exit()

# Ticker for Equinor
equinor_ticker = ["NHY.OL"]

currency = "NOK"  # Currency for Norway
file_name = "Factors/Factors Hydro.xlsx"  # File to save data

if selected_field == "ALL":
    # Fetch all fields
    selected_fields = [(field, column) for field, column in available_fields.values() if field != "ALL"]
    retrieved_data = fetch_multiple_fields(equinor_ticker, selected_fields, currency)
else:
    # Fetch a specific field
    retrieved_data = {
        expected_column: fetch_data(equinor_ticker, selected_field, expected_column, currency)
    }

# Save data
save_data_to_excel(retrieved_data, file_name, sheet_prefix="Hydro")

# Close Refinitiv session
close_refinitiv_session()


Connection successful! You are now connected to the Refinitiv Data Platform.
What do you want to fetch?
1: All Fields
2: Revenue
3: Return on Average Common Equity - %, TTM
4: Return on Average Total Assets - %, TTM
5: Price to Book Value per Share - Issue
6: Price to Net Cash Flow from Operations per Share
7: Price to Free Cash Flow per Share
8: Price to Levered Free Cash Flow per Share
9: Price to Cash Flow per Share
10: Price to Total Revenue per Share
11: Gross Profit
12: EBITDA
13: EBIT
14: Price to Tangible Book Value per Share
15: Earnings Per Share - Actual
16: Dividend Per Share - Actual
17: Book Value Per Share - Actual
74: Net Income after Tax
75: Net Income - Actual
80: Free Cash Flow - Actual
82: Unlevered Free Cash Flow
18: Company Market Cap
19: Total Equity
20: Net Property, Plant & Equipment Percentage of Total Assets
21: Goodwill - Net Percentage of Total Assets
22: Intangibles - Net Percentage of Total Assets
23: Total Fixed Assets
24: Net Operating Assets
25: Total 

## Hente ut Norske data

In [2]:
# Open Refinitiv session
open_refinitiv_session()

# Ask the user what they want to fetch
print("What do you want to fetch?")
for key, value in available_fields.items():
    print(f"{key}: {value[1]}")

try:
    selection = int(input("Select an option: "))
    if selection not in available_fields:
        print("Invalid selection. Program will exit.")
        close_refinitiv_session()
        exit()

    selected_field, expected_column = available_fields[selection]
except Exception as e:
    print(f"Error in selection: {e}. Program will exit.")
    close_refinitiv_session()
    exit()

# Les tickere fra tekstfil
try:
    with open("norwegian_tickers.txt", "r") as file:
        norwegian_tickers = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print("Ticker file not found. Please ensure 'norwegian_tickers.txt' exists.")
    close_refinitiv_session()
    exit()

currency = "NOK"  # Currency for Norway
file_name = "Factors/Factors Norway.xlsx"  # File to save data

if selected_field == "ALL":
    # Fetch all fields
    selected_fields = [(field, column) for field, column in available_fields.values() if field != "ALL"]
    retrieved_data = fetch_multiple_fields(norwegian_tickers, selected_fields, currency)
else:
    # Fetch a specific field
    retrieved_data = {
        expected_column: fetch_data(norwegian_tickers, selected_field, expected_column, currency)
    }

# Save data
save_data_to_excel(retrieved_data, file_name, sheet_prefix="Norway")

# Close Refinitiv session
close_refinitiv_session()


Connection successful! You are now connected to the Refinitiv Data Platform.
What do you want to fetch?
1: All Fields
2: Revenue
3: Return on Average Common Equity - %, TTM
4: Return on Average Total Assets - %, TTM
5: Price to Book Value per Share - Issue
6: Price to Net Cash Flow from Operations per Share
7: Price to Free Cash Flow per Share
8: Price to Levered Free Cash Flow per Share
9: Price to Cash Flow per Share
10: Price to Total Revenue per Share
11: Gross Profit
12: EBITDA
13: EBIT
14: Price to Tangible Book Value per Share
15: Earnings Per Share - Actual
16: Dividend Per Share - Actual
17: Book Value Per Share - Actual
74: Net Income after Tax
75: Net Income - Actual
80: Free Cash Flow - Actual
82: Unlevered Free Cash Flow
18: Company Market Cap
19: Total Equity
20: Net Property, Plant & Equipment Percentage of Total Assets
21: Goodwill - Net Percentage of Total Assets
22: Intangibles - Net Percentage of Total Assets
23: Total Fixed Assets
24: Net Operating Assets
25: Total 

## Sverige


In [ ]:
# Open Refinitiv session
open_refinitiv_session()

# Ask the user what they want to fetch
print("What do you want to fetch?")
for key, value in available_fields.items():
    print(f"{key}: {value[1]}")

try:
    selection = int(input("Select an option: "))
    if selection not in available_fields:
        print("Invalid selection. Program will exit.")
        close_refinitiv_session()
        exit()

    selected_field, expected_column = available_fields[selection]
except Exception as e:
    print(f"Error in selection: {e}. Program will exit.")
    close_refinitiv_session()
    exit()

# Les tickere fra tekstfil
try:
    with open("swedish_tickers.txt", "r") as file:
        swedish_tickers = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print("Ticker file not found. Please ensure 'swedish_tickers.txt' exists.")
    close_refinitiv_session()
    exit()

currency = "EUR"  # Currency for Sweden
file_name = "Factors/Factors Sweden.xlsx"  # File to save data

if selected_field == "ALL":
    # Fetch all fields
    selected_fields = [(field, column) for field, column in available_fields.values() if field != "ALL"]
    retrieved_data = fetch_multiple_fields(swedish_tickers, selected_fields, currency)
else:
    # Fetch a specific field
    retrieved_data = {
        expected_column: fetch_data(swedish_tickers, selected_field, expected_column, currency)
    }

# Save data
save_data_to_excel(retrieved_data, file_name, sheet_prefix="Sweden")

# Close Refinitiv session
close_refinitiv_session()


## Danmark

In [4]:
# Open Refinitiv session
open_refinitiv_session()

# Ask the user what they want to fetch
print("What do you want to fetch?")
for key, value in available_fields.items():
    print(f"{key}: {value[1]}")

try:
    selection = int(input("Select an option: "))
    if selection not in available_fields:
        print("Invalid selection. Program will exit.")
        close_refinitiv_session()
        exit()

    selected_field, expected_column = available_fields[selection]
except Exception as e:
    print(f"Error in selection: {e}. Program will exit.")
    close_refinitiv_session()
    exit()

# Les tickere fra tekstfil
try:
    with open("danish_tickers.txt", "r") as file:
        danish_tickers = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print("Ticker file not found. Please ensure 'danish_tickers.txt' exists.")
    close_refinitiv_session()
    exit()

currency = "DKK"  # Currency for Denmark
file_name = "Factors/Factors Denmark.xlsx"  # File to save data

if selected_field == "ALL":
    # Fetch all fields
    selected_fields = [(field, column) for field, column in available_fields.values() if field != "ALL"]
    retrieved_data = fetch_multiple_fields(danish_tickers, selected_fields, currency)
else:
    # Fetch a specific field
    retrieved_data = {
        expected_column: fetch_data(danish_tickers, selected_field, expected_column, currency)
    }

# Save data
save_data_to_excel(retrieved_data, file_name, sheet_prefix="Denmark")

# Close Refinitiv session
close_refinitiv_session()


Connection successful! You are now connected to the Refinitiv Data Platform.
What do you want to fetch?
1: All Fields
2: Revenue
3: Return on Average Common Equity - %, TTM
4: Return on Average Total Assets - %, TTM
5: Price to Book Value per Share - Issue
6: Price to Net Cash Flow from Operations per Share
7: Price to Free Cash Flow per Share
8: Price to Levered Free Cash Flow per Share
9: Price to Cash Flow per Share
10: Price to Total Revenue per Share
11: Gross Profit
12: EBITDA
13: EBIT
14: Price to Tangible Book Value per Share
15: Earnings Per Share - Actual
16: Dividend Per Share - Actual
17: Book Value Per Share - Actual
74: Net Income after Tax
75: Net Income - Actual
80: Free Cash Flow - Actual
82: Unlevered Free Cash Flow
18: Company Market Cap
19: Total Equity
20: Net Property, Plant & Equipment Percentage of Total Assets
21: Goodwill - Net Percentage of Total Assets
22: Intangibles - Net Percentage of Total Assets
23: Total Fixed Assets
24: Net Operating Assets
25: Total 

## Finland

In [64]:
# Open Refinitiv session
open_refinitiv_session()

# Ask the user what they want to fetch
print("What do you want to fetch?")
for key, value in available_fields.items():
    print(f"{key}: {value[1]}")

try:
    selection = int(input("Select an option: "))
    if selection not in available_fields:
        print("Invalid selection. Program will exit.")
        close_refinitiv_session()
        exit()

    selected_field, expected_column = available_fields[selection]
except Exception as e:
    print(f"Error in selection: {e}. Program will exit.")
    close_refinitiv_session()
    exit()

# Les tickere fra tekstfil
try:
    with open("finnish_tickers.txt", "r") as file:
        finnish_tickers = [line.strip() for line in file.readlines() if line.strip()]
except FileNotFoundError:
    print("Ticker file not found. Please ensure 'finnish_tickers.txt' exists.")
    close_refinitiv_session()
    exit()

currency = "EUR"  # Currency for Finland
file_name = "Factors/Factors Finland.xlsx"  # File to save data

if selected_field == "ALL":
    # Fetch all fields
    selected_fields = [(field, column) for field, column in available_fields.values() if field != "ALL"]
    retrieved_data = fetch_multiple_fields(finnish_tickers, selected_fields, currency)
else:
    # Fetch a specific field
    retrieved_data = {
        expected_column: fetch_data(finnish_tickers, selected_field, expected_column, currency)
    }

# Save data
save_data_to_excel(retrieved_data, file_name, sheet_prefix="Finland")

# Close Refinitiv session
close_refinitiv_session()


Connection successful! You are now connected to the Refinitiv Data Platform.
What do you want to fetch?
1: All Fields
2: Company Market Cap
3: Revenue
4: Total Equity
5: Return on Average Common Equity - %, TTM
6: Return on Average Total Assets - %, TTM
7: Price to Book Value per Share - Issue
8: Cash & Cash Equivalents - Total Percentage of Total Assets
9: Total Cash & ST Investments Percentage of Total Assets
10: Receivables Percentage of Total Assets
11: Inventory Percentage of Total Assets
12: Current Ratio
13: Current Ratio - Point of Period Difference
14: Quick Ratio
15: Quick Ratio - Point of Period Difference
16: Short-Term Debt Current Portion of Long-Term Debt
17: Working Capital Percentage of Long-Term Capital
18: Working Capital to Total Assets
19: Working Capital Year over Year Average
20: Net Property, Plant & Equipment Percentage of Total Assets
21: Goodwill - Net Percentage of Total Assets
22: Intangibles - Net Percentage of Total Assets
23: Retained Earnings to Total A

In [ ]:
import pandas as pd

# 1. Les inn CSV-filen
# Erstatt 'faktordata.csv' med navnet på filen din
csv_file = 'faktordata.csv'
data = pd.read_csv(csv_file)

# 2. Fjern prosenttegn og konverter til desimaltall
# Iterer gjennom kolonner og fjern prosenttegn hvis relevant
for col in data.columns:
    if data[col].dtype == 'object':  # Bare kolonner med tekst
        data[col] = data[col].str.replace('%', '', regex=True)  # Fjern prosenttegn
        data[col] = pd.to_numeric(data[col], errors='coerce')  # Konverter til numerisk (NaN hvis ikke mulig)

# 3. Håndter datoformatering
# Hvis du har en datokolonne, konverter den til datetime-format
if 'Date' in data.columns:  # Erstatt med det faktiske navnet på datokolonnen din
    data['Date'] = pd.to_datetime(data['Date'], errors='coerce')

# 4. Håndter manglende verdier
# Fyll inn manglende verdier (her brukes gjennomsnitt som eksempel)
data = data.fillna(data.mean())

# 5. Lagre som Stata (.dta)
stata_file = 'faktordata.dta'
data.to_stata(stata_file, write_index=False)

print(f"Datafilen er renset og lagret som {stata_file}.")
